In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib
import os
import os.path
from time import sleep
from urllib.error import HTTPError
from tqdm.auto import tqdm
import requests
import io
import csv

In [2]:
def get_lookup_table():
    print('Fetching player lookup table.')
    url = "https://raw.githubusercontent.com/chadwickbureau/register/master/data/people.csv"
    s=requests.get(url).content
    table = pd.read_csv(io.StringIO(s.decode('utf-8')), dtype={'key_sr_nfl': object, 'key_sr_nba': object, 'key_sr_nhl': object})
    #subset columns
    cols_to_keep = ['name_last','name_first','key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs', 'mlb_played_first','mlb_played_last']
    table = table[cols_to_keep]
    #make these lowercase to avoid capitalization mistakes when searching
    table['name_last'] = table['name_last'].str.lower()
    table['name_first'] = table['name_first'].str.lower()
    # Pandas cannot handle NaNs in integer columns. We need IDs to be ints for successful queries in statcast, etc. 
    # Workaround: replace ID NaNs with -1, then convert columns to integers. User will have to understand that -1 is not a valid ID. 
    table[['key_mlbam', 'key_fangraphs']] = table[['key_mlbam', 'key_fangraphs']].fillna(-1)
    table[['key_mlbam', 'key_fangraphs']] = table[['key_mlbam', 'key_fangraphs']].astype(int) # originally returned as floats which is wrong
    return table


def playerid_lookup(last, first=None):
    # force input strings to lowercase
    last = last.lower()
    if first:
        first = first.lower()
    table = get_lookup_table()
    if first is None:
        results = table.loc[table['name_last']==last]
    else:
        results = table.loc[(table['name_last']==last) & (table['name_first']==first)]
    #results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']] = results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']].astype(int) # originally returned as floats which is wrong
    results = results.reset_index().drop('index', 1)
    chosen_player_key = results['key_mlbam']
    return results

In [3]:
#user_input = input("Enter desired pitcher: ")
user_input = 'clayton kershaw'

In [4]:
split_input = user_input.split()

In [5]:
x = playerid_lookup(split_input[1], split_input[0])
print(x)

Fetching player lookup table.
  name_last name_first  key_mlbam key_retro  key_bbref  key_fangraphs  \
0   kershaw    clayton     477132  kersc001  kershcl01           2036   

   mlb_played_first  mlb_played_last  
0            2008.0           2019.0  


In [13]:
if os.path.isfile(str(x.iat[0,1])+str(x.iat[0,0])+'.csv'):
    print ("File exist")
else:
    mlbam_id = x.iat[0,2]
    print(mlbam_id)
    url = ("https://baseballsavant.mlb.com/statcast_search/csv?all="
           "true&hfPT=FF%7CFT%7CFC%7CSI%7CFS%7CSL%7CCH%7CCU%7C&hfAB="
           "&hfBBT=&hfPR=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT="
           "R%7C&hfC=&hfSea=2019%7C2018%7C2017%7C2016%7C2015%7C2014%"
           "7C2013%7C2012%7C2011%7C2010%7C2009%7C2008%7C&hfSit=&player_type="
           "pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands="
           "&hfSA=&game_date_gt=&game_date_lt=&hfInfield=&team=&position="
           "&hfOutfield=&hfRO=&home_road=&hfFlag=&hfPull=&pitchers_lookup%5B%5D="
           "{}"
           "&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&"
           "sort_col=pitches&player_event_sort=h_launch_speed&sort_order="
           "desc&min_pas=0&type=details&").format(mlbam_id)
    myfile = requests.get(url)

    open('savant_data.csv', 'wb').write(myfile.content)
    os.rename('savant_data.csv', str(x.iat[0,1])+str(x.iat[0,0])+'.csv')

File exist


In [6]:
conn = sql.connect('statcast.db')

In [15]:
cur = conn.cursor()
cur.execute("CREATE TABLE t (pitch_type, release_speed, release_spin_rate);")

OperationalError: table t already exists

In [19]:
filename = str(x.iat[0,1])+str(x.iat[0,0])+'.csv'

with open(filename,'rb') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['col1'], i['col2'], i['col3']) for i in dr]

cur.executemany("INSERT INTO t (col1, col2, col3) VALUES (?, ?);", to_db)

Error: iterator should return strings, not bytes (did you open the file in text mode?)

In [5]:
conn.commit()
conn.close()